In [1]:
%config IPCompleter.greedy=True

In [2]:
from pprint import pprint
import folium
import geopy.distance
import math
import numpy as np
import os
import time
from datetime import datetime
from IPython import embed

In [4]:
import ee
ee.Authenticate()

Enter verification code:  4/3wGCm_VK-XZHaTnv5MbHyOY45ev2HweLDw3gO4k4GEq2GoT1VgEzuH4



Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
# constants for each satellite
synthesized_mask_name = 'cloud_mask_pixelqa'
ls5 = {
    'imagecollection_id' : 'LANDSAT/LT05/C01/T1_SR',
    'pixel_size' : 30,
    'bands' : ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
    'mask_band' : ['pixel_qa'],
    'mask_bits_clear' : 1,
    'time_field' : 'system:time_start'
}
ls8 = {
    'imagecollection_id' : 'LANDSAT/LC08/C01/T1_SR',
    'pixel_size' : 30,
    'bands' : ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
    'mask_band' : ['pixel_qa'], 
    'mask_bits_clear' : 1,
    'time_field' : 'system:time_start',
    'id_date_format' : '%Y%m%d',
    'id_date_length' : 8
}
s2 = {
    'imagecollection_id' : 'COPERNICUS/S2_SR',
    'pixel_size' : 20,
    'bands' : ['B2', 'B3', 'B4', 'B8A', 'B11', 'B12'], #check up msk and qa60 msk is 20m qa60 is 60m
    'mask_band' : ['QA60'],
#     'mask_bits_clear' : (10, 11), https://gis.stackexchange.com/questions/333883/removing-clouds-from-sentinel-2-surface-reflectance-in-google-earth-engine
    'time_field' : 'system:time_start'
}
modis = {
    'imagecollection_id' : 'MODIS/006/MCD43A4',
    'pixel_size' : 500,
    'bands' : ['Nadir_Reflectance_Band3', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6', 'Nadir_Reflectance_Band7'],
    'time_field' : 'system:time_start',
    'id_date_format' : '%Y_%m_%d',
    'id_date_length' : 10
}
image_sets = {
    'ls5' : ls5,
    'ls8' : ls8,
    's2' : s2,
    'modis' : modis
}

In [7]:
# User inputted data

# seattle for rainy boi hrs
# west_long = -123.40369004277056 - 0.25
# east_long = -123.31133622196978
# north_lat = 47.254959278137015
# south_lat = 47.19573761464784 - 0.25

# west_long = 129.22395490745654
# east_long = 129.70735334495654 - 0.3
# north_lat = -20.984313762127464 
# south_lat = -21.39916978624304 + 0.25

# straya for clear boi hrs
west_long = 133.84410607205484
east_long = west_long+(134.7422383962736 - west_long)/8 #temporary just to make a smaller area
north_lat = -22.638743081339985
south_lat = north_lat - (north_lat - (-23.52560283638598))/10 #temporary just to make a smaller area

# dates
date_range = ('2014-08-01', '2014-12-31')
# date_range = ('2017-08-25','2017-12-31')
prediction_dates = ['2014-08-17']

# dataset selections
satellite_choice = 'ls8'
overlap_fraction = 1 # how much of the minimum block size to add for the overlap. Default is one.
percent_cloudy = 5
drive_folder = 'test27aug2'
drive_folder_location = 'C:\\Users\\karan\\Google Drive'

# calculated baseline variables from user data
corner_coords = [[east_long, south_lat], [west_long, south_lat], [west_long, north_lat], [east_long, north_lat], [east_long, south_lat]]
block_size = math.floor(image_sets['modis']['pixel_size']/image_sets[satellite_choice]['pixel_size'])
# minimum tile size is 10 MODIS pixels, this following formula just accounts for how many pixels we need at minimum to have it as a multiple of block size which is a PSRFM req
min_tile_dim_px = block_size*10
min_tile_dim_km = (min_tile_dim_px * image_sets[satellite_choice]['pixel_size'])/1000
min_tile_dim_km, east_long, block_size, west_long, east_long, north_lat, south_lat, min_tile_dim_px

(4.8,
 133.9563726125822,
 16,
 133.84410607205484,
 133.9563726125822,
 -22.638743081339985,
 -22.727429056844585,
 160)

In [8]:
# Creating the geometry for tiles within the specified coordinates
# first calculate the side lengths of the region selected
x_dist = geopy.distance.geodesic([south_lat, east_long], [south_lat, west_long]).km
y_dist = geopy.distance.geodesic([north_lat, east_long], [south_lat, east_long]).km

# determine the number of tile segments to fully cover the region, rounded up to ensure overlap
# need to look at this later to ensure that each tile has enough of an overlap to ensure pixels are a multiple of block size (aka cropping)
x_tile_segments = math.floor(x_dist/min_tile_dim_km)
y_tile_segments = math.floor(y_dist/min_tile_dim_km)

# generate a list of ordered coordinates (west to east, north to south) based on the number of tiles
# creating an overlap of approx 1 block_size pixels to ensure that each tile after cropping for PSRFM will still retain some overlap
km_long = abs(east_long - west_long)/x_dist
km_lat = abs(north_lat - south_lat)/y_dist
long_overlap = ((overlap_fraction * block_size * image_sets[satellite_choice]['pixel_size'])/1000) * km_long
lat_overlap = ((overlap_fraction * block_size * image_sets[satellite_choice]['pixel_size'])/1000) * km_lat

# determining the coordinate jumps for each tile(sans overlap)
x_coord_increment = abs(east_long - west_long)/x_tile_segments + long_overlap
y_coord_increment = abs(north_lat - south_lat)/y_tile_segments + lat_overlap

# creating the lists
west_tile_coords = [east_long - (tile_no + 1) * x_coord_increment for tile_no in reversed(range(x_tile_segments))]
east_tile_coords = [west_long + (tile_no + 1) * x_coord_increment for tile_no in range(x_tile_segments)]

north_tile_coords = [south_lat + (tile_no + 1) * y_coord_increment for tile_no in reversed(range(y_tile_segments))]
south_tile_coords = [north_lat - (tile_no + 1) * y_coord_increment for tile_no in range(y_tile_segments)]

tiles = np.empty((x_tile_segments, y_tile_segments), ee.Geometry)
for col in range(x_tile_segments):
    east_coord = east_tile_coords[col]
    west_coord = west_tile_coords[col]
    for row in range(y_tile_segments):
        north_coord = north_tile_coords[row]
        south_coord = south_tile_coords[row]
        tiles[col, row] = ee.Geometry.Rectangle([west_coord, south_coord, east_coord, north_coord])
        
x_dist, y_dist, min_tile_dim_km, x_coord_increment, y_coord_increment, west_tile_coords, east_tile_coords, north_tile_coords, south_tile_coords, tiles, x_tile_segments, y_tile_segments


(11.53283046215985,
 9.821049805334617,
 4.8,
 0.060805838687350316,
 0.04867748040695321,
 [133.8347609352075, 133.89556677389484],
 [133.9049119107422, 133.96571774942953],
 [-22.63007409603068, -22.67875157643763],
 [-22.68742056174694, -22.73609804215389],
 array([[<ee.geometry.Geometry object at 0x0000023B706D97C8>,
       dtype=object),
 2,
 2)

In [9]:
# functions to add bands
# should i add the datedist one here?
# source: https://gis.stackexchange.com/questions/277059/cloud-mask-for-landsat8-on-google-earth-engine/277151
# def getQABits(image_qa, bit_start, bit_end, new_band_name):
#     pattern = 0
#     for i in range(bit_start, bit_end):
#         pattern += pow(2, i)
#     return image_qa.addBands(image_qa.select(image_sets[satellite_choice]['mask_band'], [new_band_name]) \
#                              .bitwiseAnd(pattern).rightShift(bit_start))

def getQABitsls(image_qa, clear_bit, new_band_name):
    pattern = 0
    pattern = pow(2, clear_bit)
    return image_qa.addBands(image_qa.select(['pixel_qa'], [new_band_name]).bitwiseAnd(pattern).rightShift(clear_bit).eq(0))

def getPercentageClear(imgwithmask, mask_name):
    reducedimg = imgwithmask.reduceRegion(reducer = ee.Reducer.mean())
    return imgwithmask.set('calculated_cloud', reducedimg.get(mask_name))

In [10]:
# generating arrays of the fine and coarse res imagecollections
fine_res_tiles = np.empty((x_tile_segments, y_tile_segments), ee.ImageCollection)
# array to track which dates of fine res images are used to get the MODIS images corresponding to the dates
fine_res_dates = np.empty((x_tile_segments, y_tile_segments), dtype = list)

for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        #get a collection for all images within the date range, clipped to region and filtered for cloud cover
        initial_collection = ee.ImageCollection(image_sets[satellite_choice]['imagecollection_id'])\
                                .filterBounds(tiles[col, row])\
                                .filterDate(*date_range)\
                                .map(lambda image: image.clip(tiles[col, row]))\
                                .map(lambda image: getQABitsls(image, image_sets[satellite_choice]['mask_bits_clear'], synthesized_mask_name))
        newtestcol = initial_collection.map(lambda image: getPercentageClear(image, synthesized_mask_name)).filterMetadata('calculated_cloud', 'less_than', percent_cloudy)
#         check this naming for other data collections
        id_without_date = initial_collection.getInfo()['features'][0]['id'][:-8]
        
        selected_images = []
        fine_res_dates[col][row] = []
        
        for date in prediction_dates:
#           create a field calculating the distance of images from each prediction date, and find the lowest two for each date to use for PSRFM
            initial_collection = initial_collection.map(
                lambda image: image.set(f'dateDist{date}', 
                                        ee.Number(image.get('system:time_start'))
                                        .subtract(ee.Date.millis(ee.Date(date)))
                                       )
            )
#           aggregate dates then filter an imagecollection
            first_img_before = initial_collection.filterMetadata(f'dateDist{date}', 'less_than', 0).sort(f'dateDist{date}', False).getInfo()['features'][0]
            first_img_after = initial_collection.filterMetadata(f'dateDist{date}', 'greater_than', 0).sort(f'dateDist{date}', True).getInfo()['features'][0]
            
#           if there isn't a valid pair of images don't add them anywhere
            if first_img_before != None and first_img_after != None:
                selected_images.append(first_img_before['id'])
                selected_images.append(first_img_after['id'])

    #           Insert the dates selected for PSRFM to track which MODIS images to extract later
                first_image_date = first_img_before['id'][-image_sets[satellite_choice]['id_date_length']:]
                second_image_date = first_img_after['id'][-image_sets[satellite_choice]['id_date_length']:]
#             make this a triplet
                fine_res_dates[col][row].append([date.replace('-', ''), first_image_date, second_image_date])
            else:
#                 append null to fine res dates
                fine_res_dates[col][row] = None
#note: check whether duplicate images are created
        fine_res_tiles[col, row] = ee.ImageCollection(selected_images).map(lambda image: image.clip(tiles[col, row])) \
                                     .map(lambda image: getQABitsls(image, image_sets[satellite_choice]['mask_bits_clear'], synthesized_mask_name))


# pprint((fine_res_tiles[0,0].getInfo()['features']))
# for dates in fine_res_dates[0,0]:
#     for date in dates:
#         datetime
#     print(date)
# pprint(fine_res_dates[0,0])
# pprint((coarse_res_tiles[0,0].getInfo()['features']))

In [11]:
coarse_res_tiles = np.empty((x_tile_segments, y_tile_segments), ee.ImageCollection)
for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        collection = (ee.ImageCollection(image_sets['modis']['imagecollection_id'])
                        .filterBounds(tiles[col, row])
                        .filterDate(*date_range)
                        .sort(image_sets['modis']['time_field'])
                        .map(lambda image: image.clip(tiles[col, row])))
        modis_name_format = collection.first().getInfo()['id'][:-image_sets['modis']['id_date_length']]
        selected_modis_images = []
        
        for date_set in fine_res_dates[col, row]:
            for date in date_set:
                selected_modis_images.append(modis_name_format + date[:4] + '_' + date[4:6] + '_' + date[6:])
#       ensures unique dates so no duplicates
        selected_modis_images = list(dict.fromkeys(selected_modis_images))
        coarse_res_tiles[col, row] = ee.ImageCollection(selected_modis_images).map(lambda image: image.clip(tiles[col, row]))

In [12]:
# export imagecollection tile array function
fine_res_filenames = []
fine_res_mask_filenames = []
coarse_res_filenames = []

def date_to_day_of_year(date, format='%Y%m%d'):
    date = datetime.datetime.strptime(date, format=format)
    new_year_day = datetime.datetime(year=date.year, month=1, day=1)
    return (date - new_year_day).days + 1
# pprint(ee.Image(fine_res_tiles[0, 0].toList(fine_res_tiles[0, 0].size()).get(0)).getInfo())

def export_tile_array(tile_array, satellite_name):
    folder_into = f'{drive_folder}_{satellite_name}'    
    extract_mask = False
    if satellite_name.lower().strip() != 'modis':
        extract_mask = True
    for col in range(len(tile_array)):
        for row in range(len(tile_array[col])):
            for elem in range(tile_array[col, row].size().getInfo()):
                elem_image = ee.Image(tile_array[col, row].toList(tile_array[col, row].size()).get(elem))
#                 if col == row == 0:
#                     pprint(elem_image.getInfo())
                img_date = datetime.strptime(elem_image.getInfo()['id'][-image_sets[satellite_name]['id_date_length']:], 
                                             image_sets[satellite_name]['id_date_format'])
                first_day = datetime(year = img_date.year, month = 1, day = 1)
                img_day = (img_date - first_day).days + 1
                img_date_str = img_date.strftime('%d-%b-%Y')
                img_filename = f'{satellite_name}_{col}_{row}_{img_day}_{img_date_str}'
                task = ee.batch.Export.image.toDrive(
                                        image = elem_image.select(image_sets[satellite_name]['bands']).toInt16(),
                                        region = elem_image.getInfo()['properties']['system:footprint']['coordinates'],
                                        crs = 'EPSG:32653',
                                        scale = image_sets[satellite_choice]['pixel_size'],
                                        folder = folder_into,
                                        description = img_filename)
                task.start()
                if extract_mask:
                    mask_filename = f'{satellite_name}_mask_{col}_{row}_{img_day}_{img_date_str}'
                    mask_task = ee.batch.Export.image.toDrive(
                                        image = elem_image.select(synthesized_mask_name).toUint8(),
                                        region = elem_image.getInfo()['properties']['system:footprint']['coordinates'],
                                        crs = 'EPSG:32653',
                                        scale = image_sets[satellite_choice]['pixel_size'],
                                        folder = f"{folder_into}_mask",
                                        description = mask_filename)
                    mask_task.start()
                    fine_res_filenames.append(img_filename)
                    fine_res_mask_filenames.append(mask_filename)
                else:
                    coarse_res_filenames.append(img_filename)

export_tile_array(coarse_res_tiles, 'modis')
export_tile_array(fine_res_tiles, satellite_choice)
fine_res_filenames, fine_res_mask_filenames, coarse_res_filenames

(['ls8_0_0_228_16-Aug-2014',
  'ls8_0_0_244_01-Sep-2014',
  'ls8_0_1_228_16-Aug-2014',
  'ls8_0_1_244_01-Sep-2014',
  'ls8_1_0_228_16-Aug-2014',
  'ls8_1_0_244_01-Sep-2014',
  'ls8_1_1_228_16-Aug-2014',
  'ls8_1_1_244_01-Sep-2014'],
 ['ls8_mask_0_0_228_16-Aug-2014',
  'ls8_mask_0_0_244_01-Sep-2014',
  'ls8_mask_0_1_228_16-Aug-2014',
  'ls8_mask_0_1_244_01-Sep-2014',
  'ls8_mask_1_0_228_16-Aug-2014',
  'ls8_mask_1_0_244_01-Sep-2014',
  'ls8_mask_1_1_228_16-Aug-2014',
  'ls8_mask_1_1_244_01-Sep-2014'],
 ['modis_0_0_229_17-Aug-2014',
  'modis_0_0_228_16-Aug-2014',
  'modis_0_0_244_01-Sep-2014',
  'modis_0_1_229_17-Aug-2014',
  'modis_0_1_228_16-Aug-2014',
  'modis_0_1_244_01-Sep-2014',
  'modis_1_0_229_17-Aug-2014',
  'modis_1_0_228_16-Aug-2014',
  'modis_1_0_244_01-Sep-2014',
  'modis_1_1_229_17-Aug-2014',
  'modis_1_1_228_16-Aug-2014',
  'modis_1_1_244_01-Sep-2014'])

In [13]:
# wait for a full set of images to appear locally to begin psrfm processing
# for now just check that all of the images exist in the folder
fine_res_path = f'{drive_folder_location}\\{drive_folder}_{satellite_choice}'
coarse_res_path = f'{drive_folder_location}\\{drive_folder}_modis'
fine_mask_path = f'{drive_folder_location}\\{drive_folder}_{satellite_choice}_mask'
print('started')
while not (f'{drive_folder}_modis' in os.listdir(drive_folder_location)):
    time.sleep(180)
print('coarse res image folder exists')
print(os.listdir(drive_folder_location))

while not (f'{drive_folder}_{satellite_choice}' in os.listdir(f'{drive_folder_location}')):
    time.sleep(180)
print('fine res image folder exists')
print(os.listdir(drive_folder_location))

while not (f'{drive_folder}_{satellite_choice}_mask' in os.listdir(f'{drive_folder_location}')):
    time.sleep(180)
print('fine res mask folder exists')
print(os.listdir(drive_folder_location))

# wait for all the files to arrive
while len(os.listdir(fine_res_path)) != len(fine_res_filenames):
    time.sleep(30)
while len(os.listdir(coarse_res_path)) != len(coarse_res_filenames):
    time.sleep(30)
while len(os.listdir(fine_mask_path)) != len(fine_res_mask_filenames):
    time.sleep(30)
print('Ready for PSRFM with following files:')
pprint(os.listdir(fine_res_path))
pprint(os.listdir(coarse_res_path))
pprint(os.listdir(fine_mask_path))

started
coarse res image folder exists
['.tmp.drivedownload', 'AustraliaTest', 'AustraliaTest (1)', 'desktop.ini', 'Documents', 'imagedata', 'test27aug1', 'test27aug1_ls8', 'test27aug1_ls8_mask', 'test27aug1_modis', 'test27aug2_ls8', 'test27aug2_ls8_mask', 'test27aug2_modis', 'TorkelsenImagesV2']
fine res image folder exists
['.tmp.drivedownload', 'AustraliaTest', 'AustraliaTest (1)', 'desktop.ini', 'Documents', 'imagedata', 'test27aug1', 'test27aug1_ls8', 'test27aug1_ls8_mask', 'test27aug1_modis', 'test27aug2_ls8', 'test27aug2_ls8_mask', 'test27aug2_modis', 'TorkelsenImagesV2']
fine res mask folder exists
['.tmp.drivedownload', 'AustraliaTest', 'AustraliaTest (1)', 'desktop.ini', 'Documents', 'imagedata', 'test27aug1', 'test27aug1_ls8', 'test27aug1_ls8_mask', 'test27aug1_modis', 'test27aug2_ls8', 'test27aug2_ls8_mask', 'test27aug2_modis', 'TorkelsenImagesV2']
Ready for PSRFM with following files:
['ls8_0_0_228_16-Aug-2014.tif',
 'ls8_0_0_244_01-Sep-2014.tif',
 'ls8_0_1_228_16-Aug-2014

In [ ]:
# setting up folium https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb#scrollTo=zTTNRfxyMPFe
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Set visualization parameters.
vis_params_mask = {
  'bands': [synthesized_mask_name],
  'min': 0, 
  'max': 1}
vis_params_fine = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 544, 
  'max': 2520}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        dataset = ee.Image(fine_res_tiles[col, row].toList(fine_res_tiles[col, row].size()).get(1))
#         pprint(dataset.getInfo())
        occurrence = dataset.select(synthesized_mask_name)
        my_map.add_ee_layer(occurrence, vis_params_mask, f'fineresmask[{col}, {row}]')
        
for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        dataset = ee.Image(fine_res_tiles[col, row].toList(fine_res_tiles[col, row].size()).get(1))
#         pprint(dataset.getInfo()['properties'])
        occurrence = dataset.select(['B4', 'B3', 'B2'])
        my_map.add_ee_layer(occurrence, vis_params_fine, f'fineres[{col}, {row}]')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
# plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)